In [38]:
!pip install mrob

In [39]:
import mrob
import numpy as np

In [40]:
mrob.__version__

'0.0.8'

# Graph SLAM, using a library
in this seminar, we will show some of the functionalities of using a library for solving graphSLAM. [MROB: Mobile Robotics library](https://github.com/MobileRoboticsSkoltech/mrob) is a library for general use of perception techniques: FGraphs, Rigid Body Transformation and Point Cloud alignment.

Other interesting libraries to explore are [g2o (Kumerle'2011)](https://github.com/RainerKuemmerle/g2o) and [GTSAM (Dellaert'2011)](https://gtsam.org/).

## 1 Creating a Graph
We will start by creating a graph, and then add a node. You can get familiar by using help or from the examples in mrob (see github python_examples)

In [41]:
graph = mrob.FGraph()

In [42]:
with mrob.ostream_redirect(stdout=True, stderr=True):
      graph.print()

Status of graph: 0Nodes and 0Factors.


## 2. Add the first Node $x_0$
We will add the first node to the Fgraph. Create a random initial state ($\sigma = 0.1 $) and add it to the graph. For that, use the function `add_node_pose_2d()`.

Print your graph in simplified mode and in complete form.

In [43]:
?graph.add_node_pose_2d

Docstring:
add_node_pose_2d(self: mrob.mrob.FGraph, arg0: numpy.ndarray[numpy.float64[3, 1]]) -> int

 - arguments, initial estimate (np.zeros(3)
output, node id, for later usage
Type:      method


In [44]:
mean=np.array([0,0,0])
cov=np.eye(3)*0.1
n=1

state=np.random.multivariate_normal(mean,cov,n)
node_id=graph.add_node_pose_2d(state[0])


In [45]:
?graph.print
with mrob.ostream_redirect(stdout=True, stderr=True):
      graph.print(True)

Status of graph: 1Nodes and 0Factors.
Printing NodePose2d: 0, state = 
0.486267
0.333174
0.526761
and neighbour factors 0


Docstring:
print(self: mrob.mrob.FGraph, completePrint: bool = False) -> None

By default False: does not print all the information on the Fgraph
Type:      method


## 3. Add a factor to $x_0$
Now that we have a node in the graph, we want to add the first observation. In this case it will be an anchor factor, assuming we are observing that the node is at $[0,0,0]$ with information $\Sigma_{x_0}= 10^6 I$ 

In [46]:
# ?graph.add_factor_1pose_2d
mean_obs=np.array([0,0,0])
cov_obs=np.eye(3)*1e6
n_obs=1
graph.add_factor_1pose_2d(mean_obs,node_id,cov_obs)
with mrob.ostream_redirect(stdout=True, stderr=True):
      graph.print(True)

Status of graph: 1Nodes and 1Factors.
Printing NodePose2d: 0, state = 
0.486267
0.333174
0.526761
and neighbour factors 1
Printing Factor: 0, obs= 
0
0
0
 Residuals= 
1.12856e+277
8.93169e+271
4.76485e+180 
and Information matrix
1e+06     0     0
    0 1e+06     0
    0     0 1e+06
 Calculated Jacobian = 
0 0 0
0 0 0
0 0 0
 Chi2 error = 0 and neighbour Nodes 1


## 4. Analyze the current error in the graph
For this, use the function `chi2()`, which evaluates the problem at the current point and calculates the current value of the residuals.

You can also get the current state estimate by using the function `get_estimated_state()`. Print its current value.

In [47]:
graph.chi2()

312469.0118939331

In [48]:
graph.get_estimated_state()

[array([[0.48626726],
        [0.33317408],
        [0.52676105]])]

## 5. Solve
We will use the Gauss Newton routine (default) with one iteration. For that, call the function `solve()` and recalculate the current estimate and the error.

In [49]:
?graph.solve

Docstring:
solve(self: mrob.mrob.FGraph, method: mrob.mrob.FGraph.optimMethod = <FGraph.optimMethod.GN: 0>, maxIters: int = 30, lambda: float = 1e-06, solutionTolerance: float = 0.01) -> None

Solves the corresponding FG.
Options:
 method = mrob.GN (Gauss Newton), by default option. It carries out a SINGLE iteration.
                  = mrob.LM (Levenberg-Marquard), it has several parameters:
 - marIters = 30 (by default). Only for LM
 - lambda = 1-6, LM paramter for the size of the update
 - solutionTolerance: convergence criteria.
Type:      method


In [50]:
graph.solve(maxIters=int(50))
with mrob.ostream_redirect(stdout=True, stderr=True):
    
    graph.print(True)

Status of graph: 1Nodes and 1Factors.
Printing NodePose2d: 0, state = 
5.55112e-17
          0
          0
and neighbour factors 1
Printing Factor: 0, obs= 
0
0
0
 Residuals= 
0.486267
0.333174
0.526761 
and Information matrix
1e+06     0     0
    0 1e+06     0
    0     0 1e+06
 Calculated Jacobian = 
1 0 0
0 1 0
0 0 1
 Chi2 error = 312469 and neighbour Nodes 1
